# [IAPR 2020:][iapr2020] Lab 3 ‒  Classification

**Authors:** Sven Borden, Sorya Jullien, Artur Jesslen  
**Due date:** 03.05.2020

[iapr2020]: https://github.com/LTS5/iapr-2020

## Before running this notebook, verify that you have all requirements by running this command in your terminal:
`python -m pip install numpy pandas scipy sympy tensorflow matplotlib`

## Part 1
In this part, we will study classification based on the data available in the Matlab file `classification.mat` that you will under `lab-03-data/part1`.
There are 3 data sets in this file, each one being a training set for a given class.
They are contained in variables `a`, `b` and `c`.

**Note**: we can load Matlab files using the [scipy.io] module.

[scipy.io]: https://docs.scipy.org/doc/scipy/reference/io.html

## Extract relevant data
We first need to extract the `lab-03-data.tar.gz` archive.
To this end, we use the [tarfile] module from the Python standard library.

[tarfile]: https://docs.python.org/3.6/library/tarfile.html

In [ ]:
import tarfile
import os
import scipy.io
from scipy.stats import stats
import numpy as np
import pandas as pd
# Sympy modules
from sympy import symbols
from sympy import plot_implicit
from sympy import Eq
from sympy.solvers import solve
from mpmath import log
from sympy import Matrix

# defining the files path
import matplotlib.pyplot as plt
data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-03-data'
tar_path = os.path.join(data_base_path, data_folder + '.tar.gz')
with tarfile.open(tar_path, mode='r:gz') as tar:
    tar.extractall(path=data_base_path)
    
data_part1_path = os.path.join(data_base_path, data_folder, 'part1', 'classification.mat')
matfile = scipy.io.loadmat(data_part1_path)
a = matfile['a']
b = matfile['b']
c = matfile['c']

print(a.shape, b.shape, c.shape)

### 1.1 Bayes method
Using the Bayes method, give the analytical expression of the separation curves between those three classes.
Do reasonable hypotheses about the distributions of those classes and estimate the corresponding parameters based on the given training sets.
Draw those curves on a plot, together with the training data.
For simplicity reasons, round the estimated parameters to the closest integer value.

In [ ]:
class_colors = ['red', 'green', 'blue'] #used later for plotting colors

In [ ]:
# Plot more easily n by 2 matrix (n vectors)
def separateXY(matrix):
    return matrix[:,0], matrix[:,1]

In [ ]:
# Plot raw classes
plt.figure()
for label_, cl, color_ in zip(['a','b','c'],[a, b, c], class_colors):
    x, y = separateXY(cl)
    plt.scatter(x, y, label=label_, color=color_)
    plt.legend()

As we can see, we can tell that the distributions for each class is reasonably gaussian. We also can see than each class is balanced in the number of samples. To quantify how much data are similar to gaussian distribution, we will evaluate their Skewness an Kurtosis. From this quick view, we can observe that for classes `a` and `b` the covariance matrix is similar, which is not the case with class `c`

In [ ]:
# Calculate skewness and kurtosis metrics for each dimension of each class
labels = ['A_x', 'A_y', 'B_x', 'B_y', 'C_x', 'C_y']

kurtosis_metrics = stats.kurtosis(np.concatenate([a, b, c], axis = 1), axis = 0)

skewness_metrics = stats.skew(np.concatenate([a, b, c], axis = 1), axis = 0)
pd.DataFrame(index=labels, columns=['Kurtosis', 'Skewness'], data = np.asarray([kurtosis_metrics, skewness_metrics]).T)

As we can see from the values above, we have good approximations of gaussian values as all the kurtosis and skewness values are near to zero. 

Bayes rule for a class defined by the points $X = [x,y]$ :

$ P(w_i|X)=\frac{p(X|w_i)P(w_i)}{p(X)} $

The discriminant function ($f$ a monotonnally increasing function) :

$g_i(X)=f(P(w_i|X))$

The decision surface is given by:

$g_{ij}=g_i(X)-g_j(X)=0$

For mean and covariance matrix:

$
\mu_x=\cfrac{1}{n}\sum_{j=1}^{n}{x}
$

$
\mu_y=\cfrac{1}{n}\sum_{j=1}^{n}{y}
$

$\Sigma ＝ \cfrac{1}{n}(X-\hat\mu)^T(X-\hat\mu) \quad$

Since the 3 classes are modeled as Gaussians and have the same priori probabilities because they contain the same amount of data points, the discriminant function can be given as:

$g_i(X)=-\frac{1}{2}(X-\mu_i)^T\Sigma_i^{-1}(X-\mu_i)+C$

If 2 classes have the same covariance matrix (a and b in our case), the decision curve can be defined as:

$g_i(X)=w_i^TX+w_{i0}=(\Sigma^{-1} \mu_i)^TX+ln(P(w_i)-\frac{1}{2}\mu_i^T \Sigma^{-1}\mu_i$

In a more normal form, the decision curve can be defined as:

$g_{ij}(X) = w^T(X - X_{0})$ with $w = \Sigma^{-1}(\mu_i - \mu_j)$

In [ ]:
# Calculate parameters of Gaussian
def get_coefficient(data):
    #mean
    temp_mean = data.mean(axis=0)

    #covariance
    temp_len = len(data)
    temp_sig = (np.dot((data-temp_mean).T, data-temp_mean)/temp_len)
    
    # in the end we round the parameters to the closest integer
    return temp_mean.round(), temp_sig.round()

In [ ]:
# Mean and covariance for each class
means = []
sigs = []
i = 0
for label, cl in zip(['a','b','c'], [a,b,c]):
    mean, sig = get_coefficient(cl)
    means.append(mean)
    sigs.append(sig)
    
    print('mean for {} : {}'.format(label, mean))
    print('covariance matrix for {} :\n {}\n'.format(label, sig))

Discriminant Hypotheses: We find that the covariance matrix $\Sigma$ from classes `a` and`b` are identical. In theory, this means the quadratic term will disappear from the definition curve, the constant term will also be removed. So we end up with a linear function which is able to classify dataset `a` from dataset `b`. 

In [ ]:
# Calculate the coefficient of quadratic terms, primary terms and constant term 
# of x and y according to the equation above
# in the end we round the coefficient to the closest integer
def get_discriminant_curve_coeff(data_1, data_2):
    # mean and covariance for each data
    mean_1, sig_1 = get_coefficient(data_1)
    mean_2, sig_2 = get_coefficient(data_2)
    
    # separation of mean and covariance values along the x and y axis
    mean_11 , mean_12 = mean_1[0] , mean_1[1]
    mean_21 , mean_22 = mean_2[0] , mean_2[1]
    
    sig_11 , sig_12 = sig_1[0][0] , sig_1[1][1]
    sig_21 , sig_22 = sig_2[0][0] , sig_2[1][1]
    
    # coefficient of quadratic terms x^2, y^2
    coef_x_2 = (sig_21-sig_11) * sig_12 * sig_22
    coef_y_2 = (sig_22-sig_12) * sig_11*sig_21
    
    # coefficient of primary terms x, y
    coef_x_1 = (-2 * mean_11 * sig_21 + 2 * mean_21 * sig_11) * sig_12 * sig_22
    coef_y_1 = (-2 * mean_12 * sig_22 + 2 * mean_22 * sig_12) * sig_11 * sig_21
    
    # coefficient of constant term
    coef_constant_1 =  mean_11 * mean_11 * sig_12 * sig_21 * sig_22 + mean_12 * mean_12 * sig_11 * sig_21 * sig_22
    coef_constant_2 = -mean_21 * mean_21 * sig_11 * sig_12 * sig_22 - mean_22 * mean_22 * sig_11 * sig_12 * sig_21
    
    coef_constant = coef_constant_1 + coef_constant_2

    # divide all coefficients by the smallest one to have small coefficient values while keeping the same ratio
    coeff_array = np.array([coef_x_2, coef_x_1, coef_y_2, coef_y_1, coef_constant])

    coeff_array /= min(abs(coeff_array[np.flatnonzero(coeff_array)]))
    
    return np.round(coeff_array)

In [ ]:
def get_curve(coeff, x_range, y_range):
    return coeff[0]*x_range*x_range + coeff[1]*x_range + coeff[2]*y_range*y_range + coeff[3]*y_range + coeff[4]

In [ ]:
# axis plot initialization
x_range = np.arange(-10, 10, 0.1)
y_range = np.arange(-6, 6, 0.1)
x_range, y_range = np.meshgrid(x_range, y_range)

# compute the coefficients of the discriminant curves between 2 classes
coeff_ab = get_discriminant_curve_coeff(a,b)
coeff_ac = get_discriminant_curve_coeff(a,c)
coeff_bc = get_discriminant_curve_coeff(b,c)

# compute the curve between 2 classes using the coefficients computed above
curve_ab = get_curve(coeff_ab, x_range, y_range)
curve_ac = get_curve(coeff_ac, x_range, y_range)
curve_bc = get_curve(coeff_bc, x_range, y_range)

# plot the classes 
fig, ax = plt.subplots(figsize=(12, 12))
for label_, cl, color_ in zip(['a', 'b', 'c'], [a, b, c], class_colors):
    x, y = separateXY(cl)
    ax.scatter(x, y, label=label_, color=color_)
    
# plot the classification curves
for label_, con, color_ in zip(['AB', 'AC', 'BC'], [curve_ab, curve_ac, curve_bc], ['black','black','black']):
    contour_item = ax.contour(x_range, y_range, con, 0, linestyles='dashed', colors=color_)
    contour_item.levels = [label_ for val in contour_item.levels]
    ax.clabel(contour_item, contour_item.levels, inline=True, fontsize=20, colors='black')
plt.legend()
plt.show()

### 1.2 Mahalanobis distance
For classes `a` and `b`, give the expression of the Mahalanobis distance used to classify a point in class `a` or `b`, and verify the obtained classification, in comparison with the "complete" Bayes classification, for a few points of the plane.

According to the Bayesian classification for normal laws, if the covariance matrix $\Sigma$ is not diagoal, the most probable class is the one which minimizes the Mahalanobis distance.

Mahalanobis Distance:

$d_m = \left((x-\mu_i)\Sigma^{-1}(x-\mu_i)\right)^{1/2}$

the class determined by Mahalanobis distance according to below



Class=
$
\left\{
\begin{array}{rcl}
a & & {d_a <= d_b}\\
b & & {d_a > d_b}
\end{array} \right.
$

In [ ]:
# calculate mahalanobis distance of a data point from a dataset according to the equation above
def get_mahalanobis_distance(data, new_point):
    mean , sig = get_coefficient(data)
    inv_sig = np.linalg.inv(sig)
    d_m = np.sqrt((new_point - mean).dot(inv_sig).dot((new_point - mean).T))
    return d_m

In [ ]:
# a function that reflect the performance of method of Mahalanobis distance
# input: dataset and label
# output: classification accuracy, list of data points that are correctly classified 
# and list of data points that are wrongly classified
def mahalanobis_distance_performance(new_data, label):
    total_num = len(new_data)
    correct_num = 0
    correct_list = []
    wrong_list = []
    
    for data_ in new_data:
        # computation of mahalanobis distance between data point and points of class a and b
        temp_dm_a = get_mahalanobis_distance(a, data_)
        temp_dm_b = get_mahalanobis_distance(b, data_)
        
        # classification by minimizing the mahalanobis distance
        prediction = np.argmin([temp_dm_a, temp_dm_b], axis = 0)
        
        # check if the classisification is correct
        if prediction == label:
            correct_num += 1
            correct_list.append(data_)
        else:
            wrong_list.append(data_)
            
    accuracy=correct_num/total_num
    
    return accuracy, np.array(correct_list), np.array(wrong_list)

In [ ]:
# a function that reflect the performance of method of complete bayesian
# input: dataset and label
# output: classification accuracy, list of data points that are correctly classified 
# and list of data points that are wrongly classified
def complete_bayesian_performance(new_data, label):
    total_num=len(new_data)
    correct_num=0
    correct_list=[]
    wrong_list=[]
    
    for data_ in new_data:
        temp_x = data_[0]
        temp_y = data_[1]
        
        # bayesian classification 
        prediction = get_curve(coeff_ab, temp_x, temp_y)>0
        
        # check if the classisification is correct
        if prediction == label:
            correct_num+=1
            correct_list+= [data_]
        else:
            wrong_list += [data_]
    accuracy=correct_num/total_num
    return accuracy, np.array(correct_list), np.array(wrong_list)

In [ ]:
# Comparing the classification using the Mahalanobis distance compared to the Bayesian classification
# Use of letters a and b training set to test both classification algorithm
accuracy_a_mah, a_correct_list_mah, a_wrong_list_mah = mahalanobis_distance_performance(a, 0)
accuracy_b_mah, b_correct_list_mah, b_wrong_list_mah = mahalanobis_distance_performance(b, 1)

accuracy_a_bayes, a_correct_list_bayes, a_wrong_list_bayes = complete_bayesian_performance(a, 0)
accuracy_b_bayes, b_correct_list_bayes, b_wrong_list_bayes = complete_bayesian_performance(b, 1)

#### Comparaison of with Mahalanobis distance and Bayesian Method

In [ ]:
# plot of the points of classes a and b correctly/wrongly classified using the mahalanobis distance method
fig, ax = plt.subplots(1, 2, figsize=(15,6))
ax[0].scatter(a_correct_list_mah[:, 0], a_correct_list_mah[:, 1], label = 'correctly classified a')
ax[0].scatter(a_wrong_list_mah[:, 0], a_wrong_list_mah[:, 1], label = 'prediction:b,  ground truth:a')
ax[0].scatter(b_correct_list_mah[:, 0], b_correct_list_mah[:, 1], label = 'correctly classified b')
ax[0].scatter(b_wrong_list_mah[:, 0], b_wrong_list_mah[:, 1], label = 'prediction:a,  ground truth:b')
ax[0].set_title("Classification Performance by Mahalanobis Distance")

# plot of the points of classes a and b correctly/wrongly classified using the bayesian method
ax[1].scatter(a_correct_list_bayes[:, 0], a_correct_list_bayes[:, 1], label = 'correctly classified a')
ax[1].scatter(a_wrong_list_bayes[:, 0], a_wrong_list_bayes[:, 1], label = 'prediction:b,  ground truth:a')
ax[1].scatter(b_correct_list_bayes[:, 0], b_correct_list_bayes[:, 1], label = 'correctly classified b')
ax[1].scatter(b_wrong_list_bayes[:, 0], b_wrong_list_bayes[:, 1], label = 'prediction:a,  ground truth:b')
contour_item = ax[1].contour(x_range, y_range, curve_ab, 0,linestyles='dotted', colors='black')
contour_item.levels = [' AB Separation' for val in contour_item.levels]
ax[1].clabel(contour_item, contour_item.levels, inline=True, fontsize=10, colors='black')
ax[1].set_title("Classification Performance by Bayes Method")

ax[0].legend()
ax[1].legend()
plt.show()

In [ ]:
# printing the accuracy of both methods for comparison
print('By Bayes Method') 
print('Accuracy for a data set = {}'.format(accuracy_a_bayes))
print('Accuracy for b data set = {}'.format(accuracy_b_bayes))
print('Accuracy for all data in a and b data sets = {:2f}'.format(0.5 * (accuracy_a_bayes + accuracy_b_bayes)))
print('-----------------------------------------------------------------')
print('By Mahalanobis Distance')
print('Accuracy for a data set = {}'.format(accuracy_a_mah))
print('Accuracy for b data set = {}'.format(accuracy_b_mah))
print('Accuracy for all data in a and b data sets = {:2f}'.format(0.5 * (accuracy_a_mah + accuracy_b_mah)))

According to the Bayesian classification for normal laws, if the covariance matrix $\Sigma$ is not diagonal, the most probable class is the one which minimizes the Mahalanobis distance.

The plots of classification and numerical accuracy show that mahalanobis distance and complete Bayesian classifier share the same result and performance in this case, which proves the theoretical hypothesis, along with the same prior probabilities of a and b.

## Part 2
In this part, we aim to classify digits using the complete version of MNIST digits dataset.
The dataset consists of 60'000 training images and 10'000 test images of handwritten digits.
Each image has size 28x28, and has assigned a label from zero to nine, denoting the digits value.
Given this data, your task is to construct a Multilayer Perceptron (MLP) for supervised training and classification and evaluate it on the test images.

Download the MNIST dataset (all 4 files) from http://yann.lecun.com/exdb/mnist/ under `lab-03-data/part2`.
You can then use the script provided below to extract and load training and testing images in Python.

To create an MLP you are free to choose any library.
In case you don't have any preferences, we encourage you to use the [scikit-learn] package; it is a simple, efficient and free tool for data analysis and machine learning.
In this [link][sklearn-example], you can find a basic example to see how to create and train an MLP using [scikit-learn].
Your network should have the following properties:
* Input `x`: 784-dimensional (i.e. 784 visible units representing the flattened 28x28 pixel images).
* 100 hidden units `h`.
* 10 output units `y`, i.e. the labels, with a value close to one in the i-th class representing a high probability of the input representing the digit `i`.

If you need additional examples you can borrow some code from image classification tutorials.
However, we recommend that you construct a minimal version of the network on your own to gain better insights.

[scikit-learn]: http://scikit-learn.org/stable/index.html
[sklearn-example]: http://scikit-learn.org/stable/modules/neural_networks_supervised.html

### 2.1 Dataset loading
Here we first declare the methods `extract_data` and `extract_labels` so that we can reuse them later in the code.
Then we extract both the data and corresponding labels, and plot randomly some images and corresponding labels of the training set.

In [ ]:
%matplotlib inline
import gzip
import numpy as np
import matplotlib.pyplot as plt

#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from sklearn.metrics import confusion_matrix

def extract_data(filename, image_shape, image_number):
    '''extract the data from a file '''
    with gzip.open(filename) as bytestream:
        bytestream.read(16)
        buf = bytestream.read(np.prod(image_shape) * image_number)
        data = np.frombuffer(buf, dtype=np.uint8).astype(np.float32)
        data = data.reshape(image_number, image_shape[0], image_shape[1])
    return data

def extract_labels(filename, image_number):
    '''extract the labels from a file'''
    with gzip.open(filename) as bytestream:
        bytestream.read(8)
        buf = bytestream.read(1 * image_number)
        labels = np.frombuffer(buf, dtype=np.uint8).astype(np.int64)
    return labels

In [ ]:
tf.__version__
tf_version, _, _ = tf.__version__.split(".")

if int(tf_version) < 2:
    print("Please updgrade your version of tensorflow by using following command `pip install --upgrade tensorflow`")

In [ ]:
# Display the openened files
image_shape = (28, 28)
train_set_size = 60000
test_set_size = 10000

# defining the files path
data_base_path = os.path.join(os.pardir, 'data')
data_folder = 'lab-03-data'
data_part2_folder = os.path.join(data_base_path, data_folder, 'part2')

train_images_path = os.path.join(data_part2_folder, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_part2_folder, 'train-labels-idx1-ubyte.gz')
test_images_path = os.path.join(data_part2_folder, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_part2_folder, 't10k-labels-idx1-ubyte.gz')

#extract data and labels from files
train_images_extract = extract_data(train_images_path, image_shape, train_set_size)
test_images_extract = extract_data(test_images_path, image_shape, test_set_size)
train_labels_extract = extract_labels(train_labels_path, train_set_size)
test_labels_extract = extract_labels(test_labels_path, test_set_size)

print("Training Set ", train_images_extract.shape, train_labels_extract.shape)
print("Test Set", test_images_extract.shape, test_labels_extract.shape)

In [ ]:
prng = np.random.RandomState(seed=123456789)  # seed to always re-draw the same distribution
# choose randomly 10 indexes among all images indexes
plt_ind = prng.randint(low=0, high=train_set_size, size=10)

# plot the 10 chosen images
fig, axes = plt.subplots(1, 10, figsize=(12, 3))
for ax, im, lb in zip(axes, train_images_extract[plt_ind], train_labels_extract[plt_ind]):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title(lb)

In [ ]:
image_size = 28
def reformat(dataset, labels):
    """
        Reformat the data to the one-hot and flattened mode
    """
    n_dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)

    # Convert to the one hot format
    n_labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)

    return n_dataset, n_labels


num_labels = 10

train_images, train_labels = reformat(train_images_extract, train_labels_extract)
test_images, test_labels = reformat(test_images_extract, test_labels_extract)

# Display the files
print("Training Set - shape: ", train_images.shape, train_labels.shape)
print("Test Set - shape: ", test_images.shape, test_labels.shape)
print("Test Set labels - [1 to 5]: \n", test_labels[0:5,:])

### 2.2 MLP

In [ ]:
# Hyper-parameters
learning_rate = 0.01
num_epoch = 11
display_step = 5       # display the avg cost after this number of epochs

# Variables
batch_size = 100
num_input = 784 # image size as array
num_hidden1 = 100
num_hidden2 = 100
num_output = 10 #0-9 digit recognition = 10 labels
np.random.seed(0)

In [ ]:
def multi_layer_perceptron(x, weights, biases):
    """
    MLP model with 2 hidden layers.
    """
    hidden_layer1 = tf.add(tf.matmul(x, weights['w_h1']), biases['b_h1'])
    hidden_layer1 = tf.nn.relu(hidden_layer1)   # apply ReLU non-linearity
    hidden_layer2 = tf.add(tf.matmul(hidden_layer1, weights['w_h2']), biases['b_h2'])
    hidden_layer2 = tf.nn.relu(hidden_layer2)

    out_layer = tf.add(tf.matmul(hidden_layer2, weights['w_out']), biases['b_out'])  # NO non-linearity in the output layer


    return out_layer


In [ ]:
s = tf.InteractiveSession()
try:
    # Trainum_input data and labels
    x = tf.placeholder('float', [None, num_input])     # training data
    y = tf.placeholder('float', [None, num_output])    # labels

    # Weights and biases
    weights = {
        'w_h1' : tf.Variable(tf.random_normal([num_input, num_hidden1])),
        'w_h2' : tf.Variable(tf.random_normal([num_hidden1, num_hidden2])),
        'w_out': tf.Variable(tf.random_normal([num_hidden2, num_output]))
    }
    biases = {
        'b_h1' : tf.Variable(tf.random_normal([num_hidden1])), 
        'b_h2' : tf.Variable(tf.random_normal([num_hidden2])),
        'b_out': tf.Variable(tf.random_normal([num_output]))
    }

    # Construct the model
    model = multi_layer_perceptron(x, weights, biases)

    # Cost function and optimization
    # Cost function: measures the probability error in discrete classification, classes being mutually exclusive
    loss_func = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = y))
    # Adam optimizer: stochastic gradient descent method, based on adaptive estimation of 1st/2nd-order moments
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss_func)

    # Train and test

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    cost_all = np.array([])
    acc_all = np.array([])
    
    # Train the model
    print("Start training Multi Layer Perceptron model...")
    # go through the training set 'num_epoch' times
    for iter in range(num_epoch):
        avg_cost = 0.0
        num_batch = train_images.shape[0] // batch_size   # total number of batches
        # loop over all batches
        for nB in range(num_batch): 
            # for each batch, definition of the training set (data, labels)
            trainData = train_images[nB*batch_size:(nB+1)*batch_size]
            trainLabels = train_labels[nB*batch_size:(nB+1)*batch_size]
            tmp_cost, _ = sess.run([loss_func, optimizer], feed_dict={x: trainData, y: trainLabels})

            avg_cost = avg_cost + tmp_cost / num_batch
        
        # Test the model: prediction and accuracy
        correct_pred = tf.equal(tf.arg_max(model, 1), tf.arg_max(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, 'float'))
        acc = accuracy.eval(session = sess, feed_dict = {x: test_images, y: test_labels})

        if iter % display_step == 0:
            print('Epoch: %02d' %(iter), 'cost: ' + "{:.2f}".format(avg_cost), 'accuracy: ' + "{:.3f}".format(acc))
        cost_all = np.append(cost_all, avg_cost)
        acc_all = np.append(acc_all, acc)
    print("Training done...")


    # plot the accuracy and loss
    plt.figure(figsize = (15,5))
    plt.subplot(121)
    plt.plot(range(num_epoch), cost_all, color='r')
    plt.subplot(122)
    plt.plot(range(num_epoch), acc_all)
    plt.ylim(0,1)
    plt.show()
    
    final_pred = tf.arg_max(model, 1)
    prediction = final_pred.eval(session = sess, feed_dict = {x: test_images})
except:
    print('Error')

finally:
    s.close()

In [ ]:
prng = np.random.RandomState()
plt_ind = prng.randint(low=0, high=test_set_size, size=10)

# plot of 10 random images and of their class prediction
fig, axes = plt.subplots(1, 10, figsize=(20, 20))
for ax, im, lb, pred in zip(axes, test_images_extract[plt_ind], test_labels_extract[plt_ind], prediction[plt_ind]):
    ax.imshow(im, cmap='gray')
    ax.axis('off')
    ax.set_title("Pred:{} (True:{})".format(pred,lb))

We choose to implement 2 hidden layers of 100 nodes because the results were not satisfying with one hidden layer of 100 nodes. 
As we can see, the resukts are pretty satisfying as the final accuracy is higher than 90% 